In [67]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from datetime import datetime
import os
import datetime

drive.flush_and_unmount()
drive.mount('/content/drive')


Mounted at /content/drive


In [68]:
!ls drive/MyDrive/BS_696/data

 all_bene_years_raw.csv   ICD_10_CM_codes_2025.txt   MS-DRG-1023.xlsx	  pde_raw.csv
'All FFS Claims'	  inpatient_encounters.csv   MSDRG_FY_2025.txt
 diagnosis.csv		  inpatient_raw.csv	     outpatient_raw.csv


In [69]:
!ls drive/MyDrive/BS_696/data/All\ Beneficiary\ Years

#for file in os.listdir('drive/MyDrive/BS_696/data/All Beneficiary Years'):
 # print(f'drive/MyDrive/BS_696/data/All Beneficiary Years/{file}')

ls: cannot access 'drive/MyDrive/BS_696/data/All Beneficiary Years': No such file or directory


# prompt: write me code that will read all the csv's contained in the above directory (beneficiary years) and store each csv as a value in the csv_dict, with the key for each csv being bene_{i} for i in range(2015, 2026)

names_list = {}
for i in range(2015, 2026):
  names_list[i] = f'bene_{i}'

csv_dict = {}
for i in range(2015,2026):
  file_path = f'drive/MyDrive/BS_696/data/All Beneficiary Years/beneficiary_{i}.csv'
  try:
    csv_dict[names_list[i]] = pd.read_csv(file_path, sep = "|")
    print(f"Successfully loaded {file_path}")
  except FileNotFoundError:
    print(f"File not found: {file_path}")
  except pd.errors.EmptyDataError:
    print(f"Empty CSV file: {file_path}")
  except pd.errors.ParserError:
    print(f"Error parsing CSV file: {file_path}")

csv_dict["bene_2018"].iloc[:,161:172]

# prompt: Write me code to check that the columns of all the dataframes in csv_dict are the same
# add this to a utils.py file
def check_dataframe_columns(csv_dict):
  """
  Checks if all dataframes in a dictionary have the same columns.

  Args:
    csv_dict: A dictionary where keys are dataframe names and values are pandas DataFrames.

  Returns:
    True if all dataframes have the same columns, False otherwise.
    Also prints informative messages about discrepancies.
  """

  if not csv_dict:
    print("The dictionary is empty.")
    return True  # Consider an empty dictionary as having consistent columns

  first_df_name = list(csv_dict.keys())[0]
  first_df_columns = set(csv_dict[first_df_name].columns)

  for df_name, df in csv_dict.items():
    if set(df.columns) != first_df_columns:
      print(f"DataFrame '{df_name}' has different columns compared to '{first_df_name}'.")
      print(f"Columns in '{first_df_name}': {first_df_columns}")
      print(f"Columns in '{df_name}': {set(df.columns)}")
      missing_in_current = first_df_columns - set(df.columns)
      extra_in_current = set(df.columns) - first_df_columns
      if missing_in_current:
          print(f"Columns missing in '{df_name}': {missing_in_current}")
      if extra_in_current:
          print(f"Extra columns in '{df_name}': {extra_in_current}")
      return False  # Found a dataframe with different columns

  print("All dataframes have the same columns.")
  return True

#Create year variable within each dataframe for joining
for i in range(2015,2026):
  csv_dict[f"bene_{i}"]["Year"] = i

for i in range(2015,2026):
  print(csv_dict[f"bene_{i}"]["DUAL_ELGBL_MONS"].describe())

# Combine all dataframes in csv_dict
enrollment = pd.concat(csv_dict.values(), ignore_index=True)
all_bene_years_raw = pd.concat(csv_dict.values(), ignore_index=True)

all_bene_years_raw.to_csv("/content/drive/MyDrive/BS_696/data/all_bene_years_raw.csv", index=False)


In [70]:
enrollment = pd.read_csv("/content/drive/MyDrive/BS_696/data/all_bene_years_raw.csv")

<ipython-input-70-7567a5448371>:1: DtypeWarning: Columns (56,57,58,59,60,61,62,63,64,80,81,82,83,84,85,86,87,88,116,117,118,119,120,121,122,123,124,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  enrollment = pd.read_csv("/content/drive/MyDrive/BS_696/data/all_bene_years_raw.csv")


In [71]:
# add to utils.py
def describe_dataframe(df):

  column_descriptions = []
  for column in df.columns:
    column_data = df[column]
    description = {
        'column_name': column,
        'dtype': column_data.dtype,
        'non_null_count': column_data.notna().sum(),
        'null_count': column_data.isna().sum(),
        'unique_count': column_data.nunique(),
        'sample_values': list(column_data.dropna().unique())[:5],
    }
    column_descriptions.append(description)

  return pd.DataFrame(column_descriptions)

# Create the dataframe that describes the columns of inpatient

enrollment_desc = describe_dataframe(enrollment)
enrollment_desc = enrollment_desc.set_index('column_name')
enrollment_desc

,dtype,non_null_count,null_count,unique_count,sample_values
column_name,,,,,
BENE_ID,int64,86917,0,10000,"[-10000010254618, -10000010254647, -1000001025..."
STATE_CODE,int64,86917,0,51,"[1, 2, 3, 4, 6]"
COUNTY_CD,int64,86917,0,1761,"[1500, 1410, 1400, 1360, 1440]"
ZIP_CD,int64,86917,0,5855,"[36109, 35756, 36801, 35216, 35805]"
BENE_BIRTH_DT,object,86917,0,7957,"[16-Aug-1999, 14-Aug-1990, 18-Mar-1982, 24-Jul..."
...,...,...,...,...,...
CST_SHR_GRP_CD_09,float64,76917,10000,5,"[0.0, 1.0, 9.0, 2.0, 3.0]"
CST_SHR_GRP_CD_10,float64,76917,10000,5,"[0.0, 1.0, 9.0, 2.0, 3.0]"
CST_SHR_GRP_CD_11,float64,76917,10000,5,"[0.0, 1.0, 9.0, 2.0, 3.0]"


In [72]:
enrollment["BENE_ID"] = enrollment["BENE_ID"].astype(str)
enrollment["BENE_BIRTH_DT"] = pd.to_datetime(enrollment["BENE_BIRTH_DT"], format="%d-%b-%Y", errors="coerce")
enrollment["BENE_DEATH_DT"] = pd.to_datetime(enrollment["BENE_DEATH_DT"], format="%d-%b-%Y", errors="coerce")



In [73]:
enrollment["BENE_BIRTH_DT"].describe()

,BENE_BIRTH_DT
count,86917
mean,1960-12-06 09:13:29.282418816
min,1912-03-26 00:00:00
25%,1947-09-05 00:00:00
50%,1956-06-09 00:00:00
75%,1972-05-13 00:00:00
max,2023-02-16 00:00:00


In [74]:

enrollment[["BENE_ID"]+[col for col in enrollment.columns if col.startswith("DUAL")] + ["Year"]][enrollment["BENE_ID"] == str(-10000010254653)]



,BENE_ID,DUAL_ELGBL_MONS,DUAL_STUS_CD_01,DUAL_STUS_CD_02,DUAL_STUS_CD_03,DUAL_STUS_CD_04,DUAL_STUS_CD_05,DUAL_STUS_CD_06,DUAL_STUS_CD_07,DUAL_STUS_CD_08,DUAL_STUS_CD_09,DUAL_STUS_CD_10,DUAL_STUS_CD_11,DUAL_STUS_CD_12,Year
2,-10000010254653,0,06,06,06,06,06,06,06,06,06,06,06,06,2015
5977,-10000010254653,0,08,08,08,08,08,08,08,08,08,08,08,08,2016
12265,-10000010254653,0,02,02,02,02,02,02,02,02,02,02,02,02,2017
18878,-10000010254653,0,06,06,06,06,06,06,06,06,06,06,06,06,2018
25880,-10000010254653,0,03,03,03,03,03,03,03,03,03,03,03,03,2019
33326,-10000010254653,0,02,02,02,02,02,02,02,02,02,02,02,02,2020
41163,-10000010254653,0,06,06,06,06,06,06,06,06,06,06,06,06,2021
49409,-10000010254653,0,02,02,02,02,02,02,02,02,02,02,02,02,2022
58080,-10000010254653,0,08,08,08,08,08,08,08,08,08,08,08,08,2023
67259,-10000010254653,0,02,02,02,02,02,02,02,02,02,02,02,02,2024


In [75]:
enrollment.drop(columns = [col for col in enrollment.columns if col.startswith("CST_SHR")], inplace=True)


In [76]:
enrollment[[col for col in enrollment.columns if col.startswith("HMO")]]

,HMO_IND_01,HMO_IND_02,HMO_IND_03,HMO_IND_04,HMO_IND_05,HMO_IND_06,HMO_IND_07,HMO_IND_08,HMO_IND_09,HMO_IND_10,HMO_IND_11,HMO_IND_12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
86912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
enrollment_desc.loc[[row for row in enrollment_desc.index if row.startswith("HMO")], :]

,dtype,non_null_count,null_count,unique_count,sample_values
column_name,,,,,
HMO_IND_01,float64,0,86917,0,[]
HMO_IND_02,float64,0,86917,0,[]
HMO_IND_03,float64,0,86917,0,[]
HMO_IND_04,float64,0,86917,0,[]
HMO_IND_05,float64,0,86917,0,[]
HMO_IND_06,float64,0,86917,0,[]
HMO_IND_07,float64,0,86917,0,[]
HMO_IND_08,float64,0,86917,0,[]
HMO_IND_09,float64,0,86917,0,[]


In [78]:
enrollment.drop(columns= [col for col in enrollment.columns if col.startswith("HMO")], inplace=True)

In [79]:
enrollment_desc.loc[[row for row in enrollment_desc.index if row.startswith("MDCR_ENT")], :]

,dtype,non_null_count,null_count,unique_count,sample_values
column_name,,,,,
MDCR_ENTLMT_BUYIN_IND_01,object,86917,0,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_02,object,86917,0,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_03,object,86917,0,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_04,object,76917,10000,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_05,object,76917,10000,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_06,object,76917,10000,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_07,object,76917,10000,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_08,object,76917,10000,3,"[3, C, 1]"
MDCR_ENTLMT_BUYIN_IND_09,object,76917,10000,3,"[3, C, 1]"


In [80]:
# Create a list of column names
mdcr_columns = [f"MDCR_ENTLMT_BUYIN_IND_{i:02}" for i in range(1, 13)]

# Initialize the condition with the comparison of the first two columns
condition = (enrollment[mdcr_columns[0]] == enrollment[mdcr_columns[1]])

# Iterate through the remaining columns and add comparisons using &
for i in range(2, len(mdcr_columns)):
    condition &= (enrollment[mdcr_columns[i-1]] == enrollment[mdcr_columns[i]])

# Use np.where to assign the flag based on the condition
enrollment['MDCR_ENTLMT_same_flag'] = np.where(condition, 1, 0)

enrollment[["BENE_ID", "Year"] + [col for col in enrollment.columns if col.startswith("MDCR_ENTLMT")]][enrollment["MDCR_ENTLMT_same_flag"] == 1]["Year"].unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [81]:
#this variable stays the same for every row throughout every year before 2025.
#I'm assuming that it will stay the same for 2025
enrollment['MDCR_ENTLMT_BUYIN_IND_year'] = enrollment['MDCR_ENTLMT_BUYIN_IND_01']

In [82]:
enrollment.drop(columns = ["MDCR_ENTLMT_same_flag"], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("MDCR_ENTLMT")], inplace=True)

In [83]:
enrollment_desc.loc[[row for row in enrollment_desc.index if row.startswith("MDCR_STATUS_CODE")], :]

,dtype,non_null_count,null_count,unique_count,sample_values
column_name,,,,,
MDCR_STATUS_CODE_01,int64,86917,0,5,"[20, 10, 31, 11, 21]"
MDCR_STATUS_CODE_02,int64,86917,0,5,"[20, 10, 31, 11, 21]"
MDCR_STATUS_CODE_03,int64,86917,0,5,"[20, 10, 31, 11, 21]"
MDCR_STATUS_CODE_04,float64,76917,10000,5,"[20.0, 10.0, 31.0, 11.0, 21.0]"
MDCR_STATUS_CODE_05,float64,76917,10000,5,"[20.0, 10.0, 31.0, 11.0, 21.0]"
MDCR_STATUS_CODE_06,float64,76917,10000,5,"[20.0, 10.0, 31.0, 11.0, 21.0]"
MDCR_STATUS_CODE_07,float64,76917,10000,5,"[20.0, 10.0, 31.0, 11.0, 21.0]"
MDCR_STATUS_CODE_08,float64,76917,10000,5,"[20.0, 10.0, 31.0, 11.0, 21.0]"
MDCR_STATUS_CODE_09,float64,76917,10000,5,"[20.0, 10.0, 31.0, 11.0, 21.0]"


In [84]:
enrollment['MDCR_status_code_flag_age'] = np.where(enrollment['AGE_AT_END_REF_YR'] == 65, 1, 0)

enrollment[["BENE_ID", "Year", "MDCR_status_code_flag_age", "AGE_AT_END_REF_YR"] + [col for col in enrollment.columns if col.startswith("MDCR_STATUS_CODE")]][(enrollment["MDCR_status_code_flag_age"] == 1) & (enrollment['Year'] != 2025) & (enrollment["MDCR_STATUS_CODE_12" ] != 10.0)]

,BENE_ID,Year,MDCR_status_code_flag_age,AGE_AT_END_REF_YR,MDCR_STATUS_CODE_01,MDCR_STATUS_CODE_02,MDCR_STATUS_CODE_03,MDCR_STATUS_CODE_04,MDCR_STATUS_CODE_05,MDCR_STATUS_CODE_06,MDCR_STATUS_CODE_07,MDCR_STATUS_CODE_08,MDCR_STATUS_CODE_09,MDCR_STATUS_CODE_10,MDCR_STATUS_CODE_11,MDCR_STATUS_CODE_12
978,-10000010259657,2015,1,65,31,11,11,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
1141,-10000010260662,2015,1,65,21,21,21,21.0,21.0,21.0,21.0,21.0,11.0,11.0,11.0,11.0
1566,-10000010263045,2015,1,65,21,21,21,21.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
1599,-10000010263203,2015,1,65,31,31,11,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
1663,-10000010263559,2015,1,65,21,21,11,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73982,-10000010277368,2024,1,65,21,21,21,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,11.0
74121,-10000010277857,2024,1,65,21,21,21,21.0,21.0,21.0,11.0,11.0,11.0,11.0,11.0,11.0
74648,-10000010279876,2024,1,65,31,31,31,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
75787,-10000010283958,2024,1,65,21,21,21,21.0,21.0,21.0,11.0,11.0,11.0,11.0,11.0,11.0


In [85]:
# Rename the column
enrollment.rename(columns={'MDCR_status_code_flag_age': 'MDCR_status_age_change_flag'}, inplace=True)
enrollment.rename(columns={'MDCR_STATUS_CODE_01':"MDCR_status_code"}, inplace=True)
enrollment.drop(columns = [col for col in enrollment.columns if col.startswith("MDCR_STATUS_CODE")], inplace=True)

In [86]:
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("PTC_CNTRCT_ID")], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("PTC_PBP_ID")], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("PTC_PLAN_TYPE")], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("PTD_CNTRCT_ID")], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("PTD_PBP_ID")], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("PTD_SGMT_ID")], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("RDS_IND")], inplace=True)
enrollment.drop(columns=[col for col in enrollment.columns if col.startswith("DUAL_STUS_CD")], inplace=True)


In [87]:
enrollment.columns

Index(['BENE_ID', 'STATE_CODE', 'COUNTY_CD', 'ZIP_CD', 'BENE_BIRTH_DT',
       'SEX_IDENT_CD', 'BENE_RACE_CD', 'ENTLMT_RSN_ORIG', 'ENTLMT_RSN_CURR',
       'ESRD_IND', 'BENE_PTA_TRMNTN_CD', 'BENE_PTB_TRMNTN_CD', 'BENE_DEATH_DT',
       'BENE_ENROLLMT_REF_YR', 'BENE_HI_CVRAGE_TOT_MONS',
       'BENE_SMI_CVRAGE_TOT_MONS', 'BENE_STATE_BUYIN_TOT_MONS',
       'BENE_HMO_CVRAGE_TOT_MONS', 'RDS_CVRG_MONS', 'ENRL_SRC', 'SAMPLE_GROUP',
       'ENHANCED_FIVE_PERCENT_FLAG', 'CRNT_BIC_CD', 'AGE_AT_END_REF_YR',
       'COVSTART', 'DUAL_ELGBL_MONS', 'STATE_CNTY_FIPS_CD_01',
       'STATE_CNTY_FIPS_CD_02', 'STATE_CNTY_FIPS_CD_03',
       'STATE_CNTY_FIPS_CD_04', 'STATE_CNTY_FIPS_CD_05',
       'STATE_CNTY_FIPS_CD_06', 'STATE_CNTY_FIPS_CD_07',
       'STATE_CNTY_FIPS_CD_08', 'STATE_CNTY_FIPS_CD_09',
       'STATE_CNTY_FIPS_CD_10', 'STATE_CNTY_FIPS_CD_11',
       'STATE_CNTY_FIPS_CD_12', 'VALID_DEATH_DT_SW', 'RTI_RACE_CD',
       'MDCR_status_code', 'PTD_PLAN_CVRG_MONS', 'Year',
       'MDCR_status_age

In [90]:
cols_to_keep = [
    "BENE_ID", "STATE_CODE", "COUNTY_CD", "ZIP_CD", "BENE_BIRTH_DT", "SEX_IDENT_CD",
    "BENE_RACE_CD", "ESRD_IND", "BENE_ENROLLMT_REF_YR", "RTI_RACE_CD", "MDCR_status_code", "MDCR_status_age_change_flag",
    "Year"
]

enrollment = enrollment[cols_to_keep]
enrollment.info()
# All bene death dates are null (so removed it from above column list)

enrollment.to_csv("/content/drive/MyDrive/BS_696/data/enrollment.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86917 entries, 0 to 86916
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   BENE_ID                      86917 non-null  object        
 1   STATE_CODE                   86917 non-null  int64         
 2   COUNTY_CD                    86917 non-null  int64         
 3   ZIP_CD                       86917 non-null  int64         
 4   BENE_BIRTH_DT                86917 non-null  datetime64[ns]
 5   SEX_IDENT_CD                 86917 non-null  int64         
 6   BENE_RACE_CD                 86917 non-null  int64         
 7   ESRD_IND                     86917 non-null  object        
 8   BENE_ENROLLMT_REF_YR         86917 non-null  int64         
 9   RTI_RACE_CD                  86917 non-null  int64         
 10  MDCR_status_code             86917 non-null  int64         
 11  MDCR_status_age_change_flag  86917 non-nu